In [1]:
import warnings

warnings.filterwarnings('ignore')

import os
import random
import optuna
import numpy as np
import pandas as pd
import scipy.stats as st
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, StandardScaler, OneHotEncoder

In [2]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything()

In [3]:
df = pd.read_csv('./impute_set/imp3.csv')


def preprocessing(df, num, cat, numeric, category, train=True):
    df_num, df_cat = df[num], df[cat]
    if train:
        df_num = pd.DataFrame(numeric.fit_transform(df_num), columns=df_num.columns)
        df_cat = pd.DataFrame(category.fit_transform(df_cat), columns=df_cat.columns)
    else:
        df_num = pd.DataFrame(numeric.transform(df_num), columns=df_num.columns)
        df_cat = pd.DataFrame(category.transform(df_cat), columns=df_cat.columns)
    res = pd.concat([df_num, df_cat], axis=1)
    return res


def test_with_5fold(df, numeric='minmax', category='label', shuffle=True):
    f1s = []
    X = df.drop('BS3_1', axis=1)
    y = df[['BS3_1']]
    numeric_col = [
        'FEV1', 'FEV1FVC', 'age', 'BS6_3', 'BS6_2_1', 'BD1',
        '건강문해력', 'Total_slp_wk', 'EQ_5D', 'BE3_31', 'BE5_1', '질환유병기간'
    ]
    cat_col = []
    for col in X.columns:
        if col not in numeric_col:
            cat_col.append(col)

    df_num, df_cat = X[numeric_col], X[cat_col]
    if numeric == 'minmax':
        n_pre = MinMaxScaler()
    else:
        n_pre = StandardScaler()
    df_num = pd.DataFrame(n_pre.fit_transform(df_num), columns=df_num.columns)

    if category == 'label':
        c_pre = OrdinalEncoder()
        df_cat = pd.DataFrame(c_pre.fit_transform(df_cat), columns=df_cat.columns)
    else:
        c_pre = OneHotEncoder(sparse_output=False)
        df_cat = pd.DataFrame(c_pre.fit_transform(df_cat))
        # df_cat = df_cat.astype(float)

    X = pd.concat([df_num, df_cat], axis=1)

    if shuffle:
        skf = StratifiedKFold(n_splits=5, shuffle=shuffle, random_state=42)
    else:
        skf = StratifiedKFold(n_splits=5, shuffle=shuffle)
    # X.reset_index().drop('index', axis=1, inplace=True)
    # y.reset_index().drop('index', axis=1, inplace=True)
    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        # print(fold+1)
        train_X, train_y = X.iloc[train_idx], y.iloc[train_idx]
        test_X, test_y = X.iloc[test_idx], y.iloc[test_idx]
        pos = train_y.value_counts()[0] / train_y.value_counts()[1]
        print(pos)

        model = XGBClassifier(random_state=42, n_estimators=20, max_depth=4, scale_pos_weight=pos)
        model.fit(train_X, train_y)
        preds = model.predict(test_X)
        f1_ = f1_score(test_y, preds, average='macro')
        f1s.append(f1_)
    return f1s


def get_cv_results(f1s:list):
    f1s = np.array(f1s)
    mean_f1 = np.mean(f1s)
    std_f1 = np.std(f1s)
    ci95 = st.t.interval(.95, df=len(f1s)-1, loc=mean_f1, scale=std_f1/np.sqrt(len(f1s)))
    return mean_f1, std_f1, ci95

In [4]:
def objective(trial:optuna.Trial, df):
    X = df.drop('BS3_1', axis=1)
    y = df[['BS3_1']]
    numeric_col = [
        'FEV1', 'FEV1FVC', 'age', 'BS6_3', 'BS6_2_1', 'BD1',
        '건강문해력', 'Total_slp_wk', 'EQ_5D', 'BE3_31', 'BE5_1', '질환유병기간'
    ]
    cat_col = []
    for col in X.columns:
        if col not in numeric_col:
            cat_col.append(col)

    df_num, df_cat = X[numeric_col], X[cat_col]    
    df_num = pd.DataFrame(StandardScaler().fit_transform(df_num), columns=df_num.columns)
    df_cat = pd.DataFrame(OrdinalEncoder().fit_transform(df_cat), columns=df_cat.columns)
    X = pd.concat([df_num, df_cat], axis=1)
    pos = y.value_counts()[0] / y.value_counts()[1]

    scores = []
    param = {
        'n_estimators' : trial.suggest_int('n_estimators', 20, 300),
        'max_depth' : trial.suggest_int('max_depth', 4, 15),
        'learning_rate' : trial.suggest_float('learning_rate', .001, .01),
        'gamma' : trial.suggest_float('gamma', 0, 20),
        'alpha' : trial.suggest_float('alpha', 0, 10),
        'lambda' : trial.suggest_float('lambda', 1, 20),
        'min_child_weight' : trial.suggest_float('min_child_weight', 0, 10),
        'max_delta_step' : trial.suggest_int('max_delta_step', 10, 50),
        'subsample' : trial.suggest_float('subsample', .1, 1),
        'sampling_method' : trial.suggest_categorical('sampling_method', ['uniform']),
        'tree_method' : trial.suggest_categorical('tree_method', ['hist', 'approx']),
        'grow_policy' : trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        'max_bin' : trial.suggest_int('max_bin', 256, 512),
        # 'scale_pos_weight' : trial.suggest_categorical('scale_pos_weight', [1, pos]),
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        train_X, train_y = X.iloc[train_idx], y.iloc[train_idx]
        test_X, test_y = X.iloc[test_idx], y.iloc[test_idx]

        model = XGBClassifier(objective='binary:logistic', device='cuda', random_state=42, scale_pos_weight=pos, **param)    
        model.fit(train_X, train_y, verbose=False, eval_set=[(train_X, train_y),(test_X, test_y)])
        pred = model.predict(test_X)
        scores.append(f1_score(test_y, pred, average='macro'))
        
    print(f'Std: {np.std(scores):.6f}')
    return np.mean(scores)
    

In [5]:
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial:objective(trial, df), n_trials=100)
print(f'Best trial: {study.best_trial}')
print(f'Best value: {study.best_value}')
print(f'Best params: {study.best_params}')

[I 2025-05-15 16:33:14,584] A new study created in memory with name: no-name-767433ea-7cdb-4dc2-8e56-108c9bf88bd9


[I 2025-05-15 16:33:24,147] Trial 0 finished with value: 0.7261745440138979 and parameters: {'n_estimators': 190, 'max_depth': 7, 'learning_rate': 0.0011354809919757937, 'gamma': 14.504150984564479, 'alpha': 9.043262871675832, 'lambda': 5.547194695683717, 'min_child_weight': 4.639543886993586, 'max_delta_step': 39, 'subsample': 0.15960031411484232, 'sampling_method': 'uniform', 'tree_method': 'approx', 'grow_policy': 'depthwise', 'max_bin': 472}. Best is trial 0 with value: 0.7261745440138979.


Std: 0.076150


[I 2025-05-15 16:33:38,355] Trial 1 finished with value: 0.7223377061469266 and parameters: {'n_estimators': 261, 'max_depth': 15, 'learning_rate': 0.009400356294572226, 'gamma': 11.534044103901389, 'alpha': 0.563894277236805, 'lambda': 15.007998798522284, 'min_child_weight': 0.7342599363289515, 'max_delta_step': 24, 'subsample': 0.14620851694108472, 'sampling_method': 'uniform', 'tree_method': 'approx', 'grow_policy': 'depthwise', 'max_bin': 274}. Best is trial 0 with value: 0.7261745440138979.


Std: 0.057086


...

Std: 0.075680


[I 2025-05-15 16:47:35,551] Trial 98 finished with value: 0.7433470424570656 and parameters: {'n_estimators': 94, 'max_depth': 7, 'learning_rate': 0.0029165177822569116, 'gamma': 1.471610164827915, 'alpha': 5.011031474278048, 'lambda': 1.1492362843500534, 'min_child_weight': 0.6070297262137805, 'max_delta_step': 38, 'subsample': 0.21157667370975589, 'sampling_method': 'uniform', 'tree_method': 'approx', 'grow_policy': 'lossguide', 'max_bin': 329}. Best is trial 95 with value: 0.7620323607792676.


Std: 0.038584


[I 2025-05-15 16:47:43,404] Trial 99 finished with value: 0.7470264875513963 and parameters: {'n_estimators': 96, 'max_depth': 7, 'learning_rate': 0.002862836475239107, 'gamma': 1.4698713152522132, 'alpha': 4.47334300212835, 'lambda': 1.3472011137747677, 'min_child_weight': 0.27635222748542065, 'max_delta_step': 39, 'subsample': 0.21185741792555263, 'sampling_method': 'uniform', 'tree_method': 'approx', 'grow_policy': 'lossguide', 'max_bin': 328}. Best is trial 95 with value: 0.7620323607792676.


Std: 0.041337
Best trial: FrozenTrial(number=95, state=1, values=[0.7620323607792676], datetime_start=datetime.datetime(2025, 5, 15, 16, 47, 0, 207857), datetime_complete=datetime.datetime(2025, 5, 15, 16, 47, 9, 805490), params={'n_estimators': 124, 'max_depth': 7, 'learning_rate': 0.003339207651158001, 'gamma': 1.6458936755157638, 'alpha': 4.85704056557763, 'lambda': 2.5184353943919406, 'min_child_weight': 0.3826429176896404, 'max_delta_step': 38, 'subsample': 0.1563403720918737, 'sampling_method': 'uniform', 'tree_method': 'approx', 'grow_policy': 'lossguide', 'max_bin': 339}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=300, log=False, low=20, step=1), 'max_depth': IntDistribution(high=15, log=False, low=4, step=1), 'learning_rate': FloatDistribution(high=0.01, log=False, low=0.001, step=None), 'gamma': FloatDistribution(high=20.0, log=False, low=0.0, step=None), 'alpha': FloatDistribution(high=10.0, log=False, low=0.0,

In [ ]:
# [I 2025-05-15 16:47:09,805] Trial 95 finished with value: 0.7620323607792676 and parameters: {'n_estimators': 124, 'max_depth': 7, 'learning_rate': 0.003339207651158001, 'gamma': 1.6458936755157638, 'alpha': 4.85704056557763, 'lambda': 2.5184353943919406, 'min_child_weight': 0.3826429176896404, 'max_delta_step': 38, 'subsample': 0.1563403720918737, 'sampling_method': 'uniform', 'tree_method': 'approx', 'grow_policy': 'lossguide', 'max_bin': 339}. Best is trial 95 with value: 0.7620323607792676.
# Std: 0.056510